In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tifffile
from tifffile import imread
from skimage.measure import label   
from skimage.morphology import skeletonize, skeletonize_3d
from skimage import morphology, filters
from skan import Skeleton, summarize
from skan import draw
from skimage.draw import polygon
import napari
import os

def modified_max_inscribed_circle(bw, f):    
    D = ndimage.distance_transform_edt(bw)
    Rs = -np.sort(-D, axis=None)
    R = Rs[0]
    RInds = np.argsort(-D, axis=None)
    RInds = RInds[Rs >= f*R]
    [cy, cx] = np.unravel_index(RInds, D.shape)
    return R, cx, cy

def create_circular_mask(h, w, center=None, radius=None):
    if center is None: 
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

viewer = napari.Viewer()

In [2]:
# Reading the retina .tif image
im1= imread("invivodata/creminus/cre-01.tif")
image_array1 = np.array(im1)
print(image_array1.shape)
viewer.add_image(image_array1)

In [4]:
# crreating a filled polygon to draw a central bead:
shapes = viewer.layers["Shapes"].data

substitute_sprout = np.zeros((image_array1.shape[0], image_array1.shape[1]), 'uint8')
for i in shapes:
    polygon1 = []
    for ii in i:    
        polygon1.append(ii)
    for iii in polygon1:
        int_array = i.astype(int)
        listem = []
    for iiii in int_array:
        a = iiii[0]
        b = iiii[1]
        listem.append((a,b))
        listem_array = np.array(listem)        

    poly = listem_array
    rr, cc = polygon(poly[:,0], poly[:,1])
substitute_sprout[rr,cc] = 255
sprout_subs = np.array(substitute_sprout)
plt.imshow(sprout_subs)

In [5]:
R, cx, cy = modified_max_inscribed_circle(sprout_subs, f=0.9)
im_med = scipy.ndimage.median_filter(image_array1, size=(6,6)) 
sz = im_med.shape
bead = np.zeros(sz, dtype=bool)
for i in range(0, len(cx), len(cx)//2):
    # You can change radius to R*n in order to get the correct seperation of sprouts
    circ_ = create_circular_mask(sz[0], sz[1], center=(cx[i],cy[i]), radius=R*0.5)
    bead = np.logical_or(bead, circ_)

bead_removed = im_med
bead_removed[bead==1]=0

branches = bead_removed > filters.threshold_otsu(bead_removed)
plt.imshow(branches)

In [6]:
skeleton_scikit1 = skeletonize(branches)
fig, ax = plt.subplots(figsize=(10, 10))
draw.overlay_skeleton_2d(branches, skeleton_scikit1, dilate=1, axes=ax)
#plt.savefig('branch analysis results/cre+3_05R.png', dpi = 300)

In [9]:
branch_data1 = summarize(Skeleton(skeleton_scikit1))
print("Total skeleton length:", branch_data1["branch-distance"].sum())

branch_data1.hist(column='branch-distance', bins=100, range=(0,160))
#plt.savefig('branch analysis results 2022-06-08/cre+3 graph 07R.png', dpi = 300)

print("Total branch number:", len(branch_data1["branch-distance"]))
print("Number of branches longer than 40px:", len(branch_data1[branch_data1["branch-distance"]>40]))

In [21]:
# Drawing the pie chart to simplify the results:
Total_branch_number = len(branch_data1["branch-distance"])
long_branches = len(branch_data1[branch_data1["branch-distance"]>40])
short_branches = Total_branch_number-long_branches

labels = ["long_branches", "short_branches"]
y = np.array([long_branches,short_branches])
plt.pie(y, labels = labels, startangle = 90, explode=(0.1, 0.1), autopct='%1.2f%%')